In [1]:
import requests
import re
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import sys
import matplotlib.pyplot as plt
from collections import Counter
import xml.etree.ElementTree as ET
from typing import List, Dict, Any, Optional
import duckdb
import unicodedata
import importlib
import numpy as np
import csv

from pathlib import Path
import re
import pandas as pd

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
import grela_helpers

In [3]:
conn = grela_helpers.make_connection()

In [4]:
#grela_extraction_helpers.conn.close()
# importlib.reload(grela_extraction_helpers)

In [5]:
# establish connection with Google sheets
file_data = json.load(open(os.path.expanduser("../../../ServiceAccountsKey.json")))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ruland_gs = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/1od248fJpNbwMO8IVZQVO3xnzrmmTe68yS9XWajyuk5o/edit?usp=sharing")

In [6]:
lexeme_df = pd.read_parquet("../data/ruland-dictionaries.parquet")

In [7]:
lexeme_df["target_len"] = lexeme_df["target_canonical"].str.split().str.len()

In [8]:
lexeme_df = lexeme_df[lexeme_df["target_relemmatized"].str.split().str.len() < 4]

In [9]:
MAX_N = 3
lexeme_df = lexeme_df[lexeme_df["target_len"] <= MAX_N].copy()
len(lexeme_df)

2846

In [10]:
# look at a few multiword entries
lexeme_df[lexeme_df["target_len"] > 1].head(5)

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized
23,Lapis maior,[Lapis maior],lapis maior,"[Ruland1612-Lapis-maior, Ruland1612-Lapis-maio...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Lapis-m...",3,"[finde nach etlicher Meynung, die calcinirten ...","[None, None, id est, der Geist außgezogen auß ...","[finde nach etlicher Meynung, die calcinirten ...","[{'entry_id': 'Ruland1612-Lapis-maior', 'quote...",2,lapis magnus
24,Leo viridis,[Leo viridis],leo uiridis,"[Ruland1612-Leo-viridis, Ruland1612-Leo-viridi...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Leo-vir...",3,"[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[None, quorundam opinione aurum, vitriolum]","[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[{'entry_id': 'Ruland1612-Leo-viridis', 'quote...",2,leo uiridis
25,Machina tractoria,[Machina tractoria],machina tractoria,"[Ruland1612-Machina-tractoria, Ruland1612-Mach...",3,[M],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Machina...",3,"[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[None, None, None]","[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[{'entry_id': 'Ruland1612-Machina-tractoria', ...",2,machina tractoria
30,Panum aereorum species,[Panum aereorum species],panum aereorum species,"[Ruland1612-Panum-aereorum-species-,-Vorpleits...",3,[P],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Panum-a...",3,"[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...","[None, None, None]","[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...",[{'entry_id': 'Ruland1612-Panum-aereorum-speci...,3,panis aereus species
32,Pyritae aurei coloris,[Pyritae aurei coloris],pyritae aurei coloris,[Ruland1612-Pyritae-aurei-coloris-.-1.-Pyrites...,3,[P],[],[],"[{'def': '1', 'entry_id': 'Ruland1612-Pyritae-...",3,"[1, è quibus ignis elicitur, è quibus ignis no...","[1, è quibus ignis elicitur, è quibus ignis no...",[1. Pyrites qui est solidus & colorem auri tot...,[{'entry_id': 'Ruland1612-Pyritae-aurei-colori...,3,pyritus aureus color


In [11]:
# look at entries starting with mercurius
lexeme_df[lexeme_df["target_canonical"].str.startswith("mercurius")]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized
27,Mercurius,[Mercurius],mercurius,"[Ruland1612-Mercurius, Ruland1612-Mercurius, R...",3,[M],[],[],"[{'def': 'id est, sulphur.', 'entry_id': 'Rula...",3,"[id est, sulphur., est principium materiale, v...","[id est, sulphur., est principium materiale, v...","[id est, sulphur. Mercurius. Mercurius ist in ...","[{'entry_id': 'Ruland1612-Mercurius', 'quote':...",1,mercurius
96,Mercurius metallorum,[Mercurius metallorum],mercurius metallorum,"[Ruland1612-Mercurius-metallorum, Ruland1612-M...",2,[M],[],[],[{'def': 'Ist darauß die Natur der Coͤrper gez...,2,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[{'entry_id': 'Ruland1612-Mercurius-metallorum...,2,mercurius metallum
1880,Mercurius argentipigmentum,[Mercurius argentipigmentum],mercurius argentipigmentum,[Ruland1612-Mercurius-argentipigmentum],1,[M],[],[],"[{'def': 'ist Schwefel victriol, Alaun Saltz-d...",1,"[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...",[{'entry_id': 'Ruland1612-Mercurius-argentipig...,2,mercurius argentipigmentum
1881,Mercurius chambar,[Mercurius chambar],mercurius chambar,[Ruland1612-Mercurius-chambar],1,[M],[],[],"[{'def': 'ist magnesia, fuessend Coͤrper und W...",1,"[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[{'entry_id': 'Ruland1612-Mercurius-chambar', ...",2,mercurius chaambar
1882,Mercurius corallinus,[Mercurius corallinus],mercurius corallinus,[Ruland1612-Mercurius-corallinus],1,[M],[],[],[{'def': 'qui per oleum ouorum & aquas alias i...,1,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[],2,mercurius corallinus
1883,Mercurius crudus,[Mercurius crudus],mercurius crudus,[Ruland1612-Mercurius-crudus],1,[M],[],[],"[{'def': 'est is, qui nondum separatus est a s...",1,"[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[{'entry_id': 'Ruland1612-Mercurius-crudus', '...",2,mercurius crudus
1884,Mercurius crystallinus,[Mercurius crystallinus],mercurius crystallinus,"[Ruland1612-Mercurius-crystallinus,]",1,[M],[],[],[{'def': 'qui saepe sublimatus est in formam c...,1,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[],2,mercurius crystallinus
1885,Mercurius laxus,[Mercurius laxus],mercurius laxus,[Ruland1612-Mercurius-laxus],1,[M],[],[],"[{'def': 'est turbith minerale.', 'entry_id': ...",1,[est turbith minerale.],[est turbith minerale.],[est turbith minerale.],[],2,mercurius laxus
1886,Mercurius mineralium,[Mercurius mineralium],mercurius mineralium,[Ruland1612-Mercurius-mineralium],1,[M],[],[],"[{'def': 'est oleitas, vel vnctuositas de mine...",1,"[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...",[{'entry_id': 'Ruland1612-Mercurius-mineralium...,2,mercurius mineralis
1887,Mercurius regeneratus,[Mercurius regeneratus],mercurius regeneratus,[Ruland1612-Mercurius-regeneratus],1,[M],[],[],"[{'def': 'est primum ens Mercurii.', 'entry_id...",1,[est primum ens Mercurii.],[est primum ens Mercurii.],[est primum ens Mercurii.],[],2,mercurius regenero


In [12]:
# test with one specific multiword phrase
target_canonical     =  "lapis philosophorum" # "lapis philosophorum"
target_relemmatized  =  "lapis philosophus" # "lapis philosophus"
# 1) phase 1: spans only (whole GreLa, but capped)
grela_helpers.create_target_spans_table(
    conn,
    table_name="temp_target_spans_table",
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    emlap_only=False,
    max_hits=1_000_000,   # safeguard
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()

(603,)

In [186]:
# 2) phase 2: build detailed KWIC for first N hits
df = grela_helpers.kwic_from_spans(
    conn,
    spans_table="temp_target_spans_table",
    window=10,
    include_tokens=True,
    max_hits=100_000,     # or None, if you know it's safe
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [187]:
df

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352412024, 352412025]","[10, 11]","[17, 18]",emlap_100002,emlap_100002_162,emlap_100002_162,emlap_100002_162,", aquarum tam uegetabilium quam mineralium ad ...","[{'token_id': 352412014, 'token_text': ',', 'l...","Et cum hoc sufficit tibi modus generalis , aqu...","[{'token_id': 352412007, 'token_text': 'Et', '...",Pseudo-Lull,De secretis naturae,1518,1518
1,"[lapidis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352425603, 352425604]","[10, 11]","[3, 4]",emlap_100002,emlap_100002_750,emlap_100002_749,emlap_100002_750,quia ibi large continetur hec doctrina . ¶item...,"[{'token_id': 352425593, 'token_text': 'quia',...",¶item in compositione lapidis philosophorum ru...,"[{'token_id': 352425600, 'token_text': '¶item'...",Pseudo-Lull,De secretis naturae,1518,1518
2,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426859, 352426860]","[10, 11]","[3, 4]",emlap_100002,emlap_100002_800,emlap_100002_798,emlap_100002_801,"& uirtutem sue nature . ¶questio . Pater , si ...","[{'token_id': 352426849, 'token_text': '&', 'l...","Pater , si lapis philosophorum cadit in doctri...","[{'token_id': 352426856, 'token_text': 'Pater'...",Pseudo-Lull,De secretis naturae,1518,1518
3,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426881, 352426882]","[10, 11]","[14, 15]",emlap_100002,emlap_100002_801,emlap_100002_801,emlap_100002_801,"non solum ad preciosos lapides , sed et iam ad...","[{'token_id': 352426871, 'token_text': 'non', ...","Fili , hec regula non solum ad preciosos lapid...","[{'token_id': 352426867, 'token_text': 'Fili',...",Pseudo-Lull,De secretis naturae,1518,1518
4,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[353269572, 353269573]","[10, 11]","[3, 4]",emlap_100004,emlap_100004_10754,emlap_100004_10752,emlap_100004_10756,ordinem . Incipiunt capitula primi libri . Quo...,"[{'token_id': 353269562, 'token_text': 'ordine...",Quod unus est lapis philosophorum Cap .,"[{'token_id': 353269569, 'token_text': 'Quod',...","Gratarolo, Guilelmo",Verae Alchemiae,1561,1561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,"[Lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[341163674, 341163675]","[10, 11]","[1, 2]",noscemus_906982,noscemus_906982_55782,noscemus_906982_55780,noscemus_906982_55784,. 3279 4701 . 8622 corruptio in nutricibus . 7...,"[{'token_id': 341163664, 'token_text': '.', 'l...","745 Lapis philosophorum , uid .","[{'token_id': 341163673, 'token_text': '745', ...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
599,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338672111, 338672112]","[10, 11]","[1, 2]",noscemus_906984,noscemus_906984_13542,noscemus_906984_13539,noscemus_906984_13544,ib . 739 . pl. 2 . prop . de Lapide Philosopho...,"[{'token_id': 338672101, 'token_text': 'ib', '...",de Lapide Philosophorum ceu medicina uniuersal...,"[{'token_id': 338672110, 'token_text': 'de', '...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
600,"[Lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338690974, 338690975]","[10, 11]","[5, 6]",noscemus_906984,noscemus_906984_17646,noscemus_906984_17645,noscemus_906984_17649,"655 . pl. 31/2 . Hemb , Matthias , de Lapide p...","[{'token_id': 338690964, 'token_text': '655', ...","Hemb , Matthias , de Lapide philosophorum prim...","[{'token_id': 338690969, 'token_text': 'Hemb',...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
601,"[lapidis, philosophorum]",relemmatized,lemma,"[lapis, ph

In [22]:
# test with one specific multiword phrase
target_canonical     =  "aqua" # "lapis philosophorum"
target_relemmatized  =  "aqua" # "lapis philosophus"
# 1) phase 1: spans only (whole GreLa, but capped)
grela_helpers.create_target_spans_table(
    conn,
    table_name="temp_target_spans_table",
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    emlap_only=False,
    max_hits=1_000_000,   # safeguard
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [25]:
n_spans = conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()[0]
n_spans

223891

In [26]:
batch_size = 100_000
if n_spans > batch_size:
    all_chunks = []

    for start_idx in range(1, n_spans + 1, batch_size):
        end_idx = min(start_idx + batch_size - 1, n_spans)
        print(f"Processing spans {start_idx}–{end_idx}...")

        df_chunk = grela_helpers.kwic_from_spans(
            conn,
            spans_table="temp_target_spans_table",
            window=10,
            include_tokens=True,
            span_idx_min=start_idx,
            span_idx_max=end_idx,
        )

        all_chunks.append(df_chunk)

    df = pd.concat(all_chunks, ignore_index=True)
else:
    df = grela_helpers.kwic_from_spans(
            conn,
            spans_table="temp_target_spans_table",
            window=10,
            include_tokens=True,
        )

Processing spans 1–100000...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing spans 100001–200000...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing spans 200001–223891...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
len(df)

In [14]:
def safe_name(s: str | None) -> str:
    """Make a safe short filename component from a target phrase."""
    if not s or not isinstance(s, str):
        return "unknown"
    s = s.strip().lower().replace(" ", "_")
    s = re.sub(r"[^a-z0-9_]+", "", s)   # keep only safe chars
    return s or "unnamed"

In [15]:
outdir = Path("../data/large_files/emlap_ruland_instances/")
outdir.mkdir(exist_ok=True)

In [48]:
def from_entries_to_file(tc, tr, outdir):
    name = safe_name(tc)
    fname = f"{name}.parquet"
    fpath = str(outdir / fname)
    grela_helpers.create_target_spans_table(
        conn,
        table_name="temp_target_spans_table",
        target_canonical=tc,
        target_relemmatized=tr,
        emlap_only=False,
        max_hits=1_000_000,   # safeguard
    )
    batch_size = 100_000
    n_spans = conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()[0]
    if n_spans <= batch_size:
        df = grela_helpers.kwic_from_spans(
            conn,
            spans_table="temp_target_spans_table",
            window=10,
            include_tokens=True,
            out_path=fpath,
            )

In [19]:
tc = "mercurius metallorum"
tr = "mercurius metallum"
from_entries_to_file(tc, tr, outdir)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [16]:
def read_hits(fname):
    try:
        path = os.path.join(outdir, fname)
        instances = pd.read_parquet(path).to_dict("records")
    except:
        instances = []
    return instances

In [17]:
df = read_hits("mercurius_metallorum.parquet")

In [18]:
pd.DataFrame(df)

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,"[mercurio, metalla]",relemmatized,lemma,"[mercurius, metallum]","[261440562, 261440563]","[10, 11]","[9, 10]",cc_13979,cc_13979_7508,cc_13979_7507,cc_13979_7508,". Metalla quoque in Mercurium uiuum mutari , e...","[{'token_id': 261440552, 'token_text': '.', 'l...","Metalla quoque in Mercurium uiuum mutari , et ...","[{'token_id': 261440553, 'token_text': 'Metall...","Sennert, Daniel",Epitome naturalis scientiae,1572,1637
1,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[353223376, 353223377]","[10, 11]","[53, 54]",emlap_100004,emlap_100004_8365,emlap_100004_8365,emlap_100004_8366,"omnibus modis praedominatur ignis , & illud di...","[{'token_id': 353223366, 'token_text': 'omnibu...","Uiuum causat metalla , quanuis unum differt ab...","[{'token_id': 353223323, 'token_text': 'Uiuum'...","Gratarolo, Guilelmo",Verae Alchemiae,1561,1561
2,"[mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[354467395, 354467396]","[10, 11]","[53, 54]",emlap_100007,emlap_100007_1504,emlap_100007_1504,emlap_100007_1506,"omnibus moribus predominatur ignis , & illud d...","[{'token_id': 354467385, 'token_text': 'omnibu...",Uiuum causat metalla quamuis adhuc bene differ...,"[{'token_id': 354467342, 'token_text': 'Uiuum'...",None,Rosarium philosophorum,1550,1550
3,"[mercurius, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[350152161, 350152162]","[10, 11]","[28, 29]",emlap_100010,emlap_100010_470,emlap_100010_470,emlap_100010_470,"similitudine uerae calcis communis , quia arge...","[{'token_id': 350152151, 'token_text': 'simili...",Compraehendis igitur ex supradictis rationibus...,"[{'token_id': 350152133, 'token_text': 'Compra...","Bracesco, Giovanni",De Alchimia Dialogi Duo,1548,1548
4,"[mercurium, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[350154009, 350154010]","[10, 11]","[6, 7]",emlap_100010,emlap_100010_569,emlap_100010_568,emlap_100010_569,"sulphure uerba faciemus . Prius diximus , per ...","[{'token_id': 350153999, 'token_text': 'sulphu...","Prius diximus , per calcinationem , mercurium ...","[{'token_id': 350154003, 'token_text': 'Prius'...","Bracesco, Giovanni",De Alchimia Dialogi Duo,1548,1548
5,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356302332, 356302333]","[10, 11]","[54, 55]",emlap_100011,emlap_100011_3877,emlap_100011_3877,emlap_100011_3878,"omnibus modis praedominatur ignis , & illud di...","[{'token_id': 356302322, 'token_text': 'omnibu...","Uiuum causat metalla , quamuis unum differt ab...","[{'token_id': 356302278, 'token_text': 'Uiuum'...",None,De alchemia,1541,1541
6,"[Mercurii, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356161082, 356161083]","[10, 11]","[1, 2]",emlap_100016,emlap_100016_6506,emlap_100016_6504,emlap_100016_6508,Quomodo argentum uiuum in sulphur conuertatur ...,"[{'token_id': 356161072, 'token_text': 'Quomod...",137 Mercurii metallorum quomodo differant .,"[{'token_id': 356161081, 'token_text': '137', ...","Bonus, Petrus",Pretiosa margarita novella,1546,1546
7,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356165596, 356165597]","[10, 11]","[57, 58]",emlap_100022,emlap_100022_141,emlap_100022_141,emlap_100022_142,"omnibus moribus praedominatur ignis , & illud ...","[{'token_id': 356165586, 'token_text': 'omnibu...","Uiuum causat metalla , quamuis adhuc bene diff...","[{'token_id': 356165539, 'token_text': 'Uiuum'...",None,De alchimia opuscula,1550,1550
8,"[Mercurium, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356170017, 356170018]","[10, 11]","[13, 14]",emlap_100022,emlap_100022_326,emlap_100022_326,emlap_100022_326,", tunc sequeretur , quod de nouo extra primam ...","[{'token_id': 356170007, 'token_text': ','

In [26]:
lexeme_df["instance_fname"] = lexeme_df["target_canonical"].apply(lambda x: safe_name(x) + ".parquet")

In [27]:
lexeme_df[lexeme_df.duplicated(subset="instance_fname", keep=False)]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized,instance_fname


In [22]:
done = []

In [ ]:
for i, row in lexeme_df.iterrows():
    tc = row.get("target_canonical")
    tr = row.get("target_relemmatized")
    target = tc or tr
    name = safe_name(tc)
    fname = f"{name}.parquet"
    if fname not in done:
        from_entries_to_file(tc, tr, outdir)
        done.append(fname)

In [4]:
len(os.listdir(outdir))

2217

In [19]:
emlap_metadata = pd.read_csv(
        "https://raw.githubusercontent.com/CCS-ZCU/EMLAP_ETL/refs/heads/master/data/emlap_metadata.csv",
        sep=";",
    )
emlap_metadata.head(5)

,Unnamed: 0,working_title,filenames,no.,is_done,is_noscemus,if_noscemus_id,AUTHORSHIP,is_one_author,#if more than 1 author skip section and choose compendium below,...,CONTENTS,genre,subject,SOURCE OF FILE,link,source_of_file,origin_of_copy,other_notes,tokens_N,aurhor_wd
0,0,"Augurello, Chrysopoeia",100001_Augurello1515_Chrysopoeia_GB_Noscemus,100001,True,True,713324.0,NaN,True,NaN,...,NaN,didactic poem,alchemy,NaN,https://wiki.uibk.ac.at/noscemus/Chrysopoeia,GB,Noscemus,NaN,23718,NaN
1,1,"Pseudo-Lull, Secretis",100002_Pseudo-Lull1518_De secretis_naturae_MDZ...,100002,True,False,NaN,NaN,True,NaN,...,NaN,treatise,"alchemy, medicine",NaN,https://www.digitale-sammlungen.de/en/view/bsb...,MDZ,MBS,NaN,24673,NaN
2,2,"Pantheus, Ars Transmutatione",100003_Pantheus1518_Ars_Transmutationis_Metall...,100003,True,False,NaN,NaN,True,NaN,...,NaN,treatise,alchemy,NaN,https://www.google.co.uk/books/edition/Ars_Tra...,GB,BL,NaN,8646,NaN
3,3,"Anon, Vera alchemiae",100004_Anon1561_Verae_Alchemiae_MDZ_MBS,100004,True,False,NaN,NaN,True,NaN,...,NaN,"compendium, florilegium",alchemy,NaN,https://mdz-nbn-resolving.de/details:bsb10141168,MDZ,MBS,NaN,3521,NaN
4,4,"Pantheus, Voarchadumia",100005_Pantheus1530_Voarchadumia_ONB,100005,True,False,NaN,NaN,True,NaN,...,NaN,treatise,alchemy,NaN,https://data.onb.ac.at/rep/10588E49,ONB,ONB,NaN,20386,NaN


In [43]:
df = pd.DataFrame(read_hits("lapis_philosophorum.parquet"))
df

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352412024, 352412025]","[10, 11]","[17.0, 18.0]",emlap_100002,emlap_100002_162,emlap_100002_162,emlap_100002_162,", aquarum tam uegetabilium quam mineralium ad ...","[{'token_id': 352412014, 'token_text': ',', 'l...","Et cum hoc sufficit tibi modus generalis , aqu...","[{'token_id': 352412007, 'token_text': 'Et', '...",Pseudo-Lull,De secretis naturae,1518,1518
1,"[lapidis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352425603, 352425604]","[10, 11]","[3.0, 4.0]",emlap_100002,emlap_100002_750,emlap_100002_749,emlap_100002_750,quia ibi large continetur hec doctrina . ¶item...,"[{'token_id': 352425593, 'token_text': 'quia',...",¶item in compositione lapidis philosophorum ru...,"[{'token_id': 352425600, 'token_text': '¶item'...",Pseudo-Lull,De secretis naturae,1518,1518
2,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426859, 352426860]","[10, 11]","[3.0, 4.0]",emlap_100002,emlap_100002_800,emlap_100002_798,emlap_100002_801,"& uirtutem sue nature . ¶questio . Pater , si ...","[{'token_id': 352426849, 'token_text': '&', 'l...","Pater , si lapis philosophorum cadit in doctri...","[{'token_id': 352426856, 'token_text': 'Pater'...",Pseudo-Lull,De secretis naturae,1518,1518
3,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426881, 352426882]","[10, 11]","[14.0, 15.0]",emlap_100002,emlap_100002_801,emlap_100002_801,emlap_100002_801,"non solum ad preciosos lapides , sed et iam ad...","[{'token_id': 352426871, 'token_text': 'non', ...","Fili , hec regula non solum ad preciosos lapid...","[{'token_id': 352426867, 'token_text': 'Fili',...",Pseudo-Lull,De secretis naturae,1518,1518
4,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[353269572, 353269573]","[10, 11]","[3.0, 4.0]",emlap_100004,emlap_100004_10754,emlap_100004_10752,emlap_100004_10756,ordinem . Incipiunt capitula primi libri . Quo...,"[{'token_id': 353269562, 'token_text': 'ordine...",Quod unus est lapis philosophorum Cap .,"[{'token_id': 353269569, 'token_text': 'Quod',...","Gratarolo, Guilelmo",Verae Alchemiae,1561,1561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,"[Lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[341163674, 341163675]","[10, 11]","[1.0, 2.0]",noscemus_906982,noscemus_906982_55782,noscemus_906982_55780,noscemus_906982_55784,. 3279 4701 . 8622 corruptio in nutricibus . 7...,"[{'token_id': 341163664, 'token_text': '.', 'l...","745 Lapis philosophorum , uid .","[{'token_id': 341163673, 'token_text': '745', ...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
599,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338672111, 338672112]","[10, 11]","[1.0, 2.0]",noscemus_906984,noscemus_906984_13542,noscemus_906984_13539,noscemus_906984_13544,ib . 739 . pl. 2 . prop . de Lapide Philosopho...,"[{'token_id': 338672101, 'token_text': 'ib', '...",de Lapide Philosophorum ceu medicina uniuersal...,"[{'token_id': 338672110, 'token_text': 'de', '...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
600,"[Lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338690974, 338690975]","[10, 11]","[5.0, 6.0]",noscemus_906984,noscemus_906984_17646,noscemus_906984_17645,noscemus_906984_17649,"655 . pl. 31/2 . Hemb , Matthias , de Lapide p...","[{'token_id': 338690964, 'token_text': '655', ...","Hemb , Matthias , de Lapide philosophorum prim...","[{'token_id': 338690969, 'token_text': 'Hemb',...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
601,"[lapidis, philosophorum]

In [44]:
set_with_dataframe(ruland_gs.add_worksheet("casestudy_lapis_philosophorum", 1,1), df)

In [45]:
df = pd.DataFrame(read_hits("mercurius_metallorum.parquet"))
df

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,"[mercurio, metalla]",relemmatized,lemma,"[mercurius, metallum]","[261440562, 261440563]","[10, 11]","[9, 10]",cc_13979,cc_13979_7508,cc_13979_7507,cc_13979_7508,". Metalla quoque in Mercurium uiuum mutari , e...","[{'token_id': 261440552, 'token_text': '.', 'l...","Metalla quoque in Mercurium uiuum mutari , et ...","[{'token_id': 261440553, 'token_text': 'Metall...","Sennert, Daniel",Epitome naturalis scientiae,1572,1637
1,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[353223376, 353223377]","[10, 11]","[53, 54]",emlap_100004,emlap_100004_8365,emlap_100004_8365,emlap_100004_8366,"omnibus modis praedominatur ignis , & illud di...","[{'token_id': 353223366, 'token_text': 'omnibu...","Uiuum causat metalla , quanuis unum differt ab...","[{'token_id': 353223323, 'token_text': 'Uiuum'...","Gratarolo, Guilelmo",Verae Alchemiae,1561,1561
2,"[mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[354467395, 354467396]","[10, 11]","[53, 54]",emlap_100007,emlap_100007_1504,emlap_100007_1504,emlap_100007_1506,"omnibus moribus predominatur ignis , & illud d...","[{'token_id': 354467385, 'token_text': 'omnibu...",Uiuum causat metalla quamuis adhuc bene differ...,"[{'token_id': 354467342, 'token_text': 'Uiuum'...",None,Rosarium philosophorum,1550,1550
3,"[mercurius, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[350152161, 350152162]","[10, 11]","[28, 29]",emlap_100010,emlap_100010_470,emlap_100010_470,emlap_100010_470,"similitudine uerae calcis communis , quia arge...","[{'token_id': 350152151, 'token_text': 'simili...",Compraehendis igitur ex supradictis rationibus...,"[{'token_id': 350152133, 'token_text': 'Compra...","Bracesco, Giovanni",De Alchimia Dialogi Duo,1548,1548
4,"[mercurium, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[350154009, 350154010]","[10, 11]","[6, 7]",emlap_100010,emlap_100010_569,emlap_100010_568,emlap_100010_569,"sulphure uerba faciemus . Prius diximus , per ...","[{'token_id': 350153999, 'token_text': 'sulphu...","Prius diximus , per calcinationem , mercurium ...","[{'token_id': 350154003, 'token_text': 'Prius'...","Bracesco, Giovanni",De Alchimia Dialogi Duo,1548,1548
5,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356302332, 356302333]","[10, 11]","[54, 55]",emlap_100011,emlap_100011_3877,emlap_100011_3877,emlap_100011_3878,"omnibus modis praedominatur ignis , & illud di...","[{'token_id': 356302322, 'token_text': 'omnibu...","Uiuum causat metalla , quamuis unum differt ab...","[{'token_id': 356302278, 'token_text': 'Uiuum'...",None,De alchemia,1541,1541
6,"[Mercurii, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356161082, 356161083]","[10, 11]","[1, 2]",emlap_100016,emlap_100016_6506,emlap_100016_6504,emlap_100016_6508,Quomodo argentum uiuum in sulphur conuertatur ...,"[{'token_id': 356161072, 'token_text': 'Quomod...",137 Mercurii metallorum quomodo differant .,"[{'token_id': 356161081, 'token_text': '137', ...","Bonus, Petrus",Pretiosa margarita novella,1546,1546
7,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356165596, 356165597]","[10, 11]","[57, 58]",emlap_100022,emlap_100022_141,emlap_100022_141,emlap_100022_142,"omnibus moribus praedominatur ignis , & illud ...","[{'token_id': 356165586, 'token_text': 'omnibu...","Uiuum causat metalla , quamuis adhuc bene diff...","[{'token_id': 356165539, 'token_text': 'Uiuum'...",None,De alchimia opuscula,1550,1550
8,"[Mercurium, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356170017, 356170018]","[10, 11]","[13, 14]",emlap_100022,emlap_100022_326,emlap_100022_326,emlap_100022_326,", tunc sequeretur , quod de nouo extra primam ...","[{'token_id': 356170007, 'token_text': ','

In [46]:
set_with_dataframe(ruland_gs.add_worksheet("casestudy_mercurius_metallorum", 1,1), df)

In [21]:
# check instances from noscemus
df[df["grela_id"].str.startswith("noscemus")]

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
467,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308529698, 308529699]","[10, 11]","[1.0, 2.0]",noscemus_1085290,noscemus_1085290_1138,noscemus_1085290_1135,noscemus_1085290_1139,& amaro edidit . U. Laur . Gryll . de Lapide P...,"[{'token_id': 308529688, 'token_text': '&', 'l...",de Lapide Philosophorum .,"[{'token_id': 308529697, 'token_text': 'de', '...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
468,"[Lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308632543, 308632544]","[10, 11]","[18.0, 19.0]",noscemus_1085290,noscemus_1085290_15736,noscemus_1085290_15736,noscemus_1085290_15736,"uentilata & nec dum relo luta Controuersia , a...","[{'token_id': 308632533, 'token_text': 'uentil...",Ethicochymico - medicum de illa ueteri toties ...,"[{'token_id': 308632525, 'token_text': 'Ethico...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
469,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308637520, 308637521]","[10, 11]","[4.0, 5.0]",noscemus_1085290,noscemus_1085290_16346,noscemus_1085290_16345,noscemus_1085290_16347,", 1552 , in fol . Efferrarius , Monachus De La...","[{'token_id': 308637510, 'token_text': ',', 'l...","Efferrarius , Monachus De Lapide Philosophorum...","[{'token_id': 308637516, 'token_text': 'Efferr...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
470,"[Lapide, Philosopha]",relemmatized,lemma,"[lapis, philosophus]","[308642667, 308642668]","[10, 11]","[26.0, 27.0]",noscemus_1085290,noscemus_1085290_17090,noscemus_1085290_17090,noscemus_1085290_17091,que descriptae Tius praeter illas etiam in phu...,"[{'token_id': 308642657, 'token_text': 'que', ...","Qua non paucae ab Antiquioribus Theophrasto , ...","[{'token_id': 308642641, 'token_text': 'Qua', ...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
471,"[Lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308534362, 308534363]","[10, 11]","[0.0, 1.0]",noscemus_1085290,noscemus_1085290_1823,noscemus_1085290_1820,noscemus_1085290_1824,". Uenetiis , ex Offic . Farrea , 1543 . Lapis ...","[{'token_id': 308534352, 'token_text': '.', 'l...","Lapis Philosophorum , aperiens , ac in 8 .","[{'token_id': 308534362, 'token_text': 'Lapis'...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,"[Lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[341163674, 341163675]","[10, 11]","[1.0, 2.0]",noscemus_906982,noscemus_906982_55782,noscemus_906982_55780,noscemus_906982_55784,. 3279 4701 . 8622 corruptio in nutricibus . 7...,"[{'token_id': 341163664, 'token_text': '.', 'l...","745 Lapis philosophorum , uid .","[{'token_id': 341163673, 'token_text': '745', ...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
599,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338672111, 338672112]","[10, 11]","[1.0, 2.0]",noscemus_906984,noscemus_906984_13542,noscemus_906984_13539,noscemus_906984_13544,ib . 739 . pl. 2 . prop . de Lapide Philosopho...,"[{'token_id': 338672101, 'token_text': 'ib', '...",de Lapide Philosophorum ceu medicina uniuersal...,"[{'token_id': 338672110, 'token_text': 'de', '...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
600,"[Lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338690974, 338690975]","[10, 11]","[5.0, 6.0]",noscemus_906984,noscemus_906984_17646,noscem

In [22]:
# find NOSCEMUS entries included in EMLAP
noscemus_emlap_ids = [str(int(x)) for x in emlap_metadata["if_noscemus_id"].unique() if pd.notna(x)]
noscemus_emlap_ids

['713324', '26765', '597737', '971293', '688159']

In [23]:
df[df["grela_id"].apply(lambda x: (x[:8] == "noscemus") and  (x[9:] not in noscemus_emlap_ids))]

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
467,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308529698, 308529699]","[10, 11]","[1.0, 2.0]",noscemus_1085290,noscemus_1085290_1138,noscemus_1085290_1135,noscemus_1085290_1139,& amaro edidit . U. Laur . Gryll . de Lapide P...,"[{'token_id': 308529688, 'token_text': '&', 'l...",de Lapide Philosophorum .,"[{'token_id': 308529697, 'token_text': 'de', '...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
468,"[Lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308632543, 308632544]","[10, 11]","[18.0, 19.0]",noscemus_1085290,noscemus_1085290_15736,noscemus_1085290_15736,noscemus_1085290_15736,"uentilata & nec dum relo luta Controuersia , a...","[{'token_id': 308632533, 'token_text': 'uentil...",Ethicochymico - medicum de illa ueteri toties ...,"[{'token_id': 308632525, 'token_text': 'Ethico...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
469,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308637520, 308637521]","[10, 11]","[4.0, 5.0]",noscemus_1085290,noscemus_1085290_16346,noscemus_1085290_16345,noscemus_1085290_16347,", 1552 , in fol . Efferrarius , Monachus De La...","[{'token_id': 308637510, 'token_text': ',', 'l...","Efferrarius , Monachus De Lapide Philosophorum...","[{'token_id': 308637516, 'token_text': 'Efferr...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
470,"[Lapide, Philosopha]",relemmatized,lemma,"[lapis, philosophus]","[308642667, 308642668]","[10, 11]","[26.0, 27.0]",noscemus_1085290,noscemus_1085290_17090,noscemus_1085290_17090,noscemus_1085290_17091,que descriptae Tius praeter illas etiam in phu...,"[{'token_id': 308642657, 'token_text': 'que', ...","Qua non paucae ab Antiquioribus Theophrasto , ...","[{'token_id': 308642641, 'token_text': 'Qua', ...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
471,"[Lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308534362, 308534363]","[10, 11]","[0.0, 1.0]",noscemus_1085290,noscemus_1085290_1823,noscemus_1085290_1820,noscemus_1085290_1824,". Uenetiis , ex Offic . Farrea , 1543 . Lapis ...","[{'token_id': 308534352, 'token_text': '.', 'l...","Lapis Philosophorum , aperiens , ac in 8 .","[{'token_id': 308534362, 'token_text': 'Lapis'...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,"[Lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[341163674, 341163675]","[10, 11]","[1.0, 2.0]",noscemus_906982,noscemus_906982_55782,noscemus_906982_55780,noscemus_906982_55784,. 3279 4701 . 8622 corruptio in nutricibus . 7...,"[{'token_id': 341163664, 'token_text': '.', 'l...","745 Lapis philosophorum , uid .","[{'token_id': 341163673, 'token_text': '745', ...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
599,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338672111, 338672112]","[10, 11]","[1.0, 2.0]",noscemus_906984,noscemus_906984_13542,noscemus_906984_13539,noscemus_906984_13544,ib . 739 . pl. 2 . prop . de Lapide Philosopho...,"[{'token_id': 338672101, 'token_text': 'ib', '...",de Lapide Philosophorum ceu medicina uniuersal...,"[{'token_id': 338672110, 'token_text': 'de', '...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
600,"[Lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338690974, 338690975]","[10, 11]","[5.0, 6.0]",noscemus_906984,noscemus_906984_17646,noscem

In [24]:
csv_path = "../data/emlap_ruland_instances_csv/"
os.makedirs(csv_path, exist_ok=True)

def _is_missing(v) -> bool:
    # safe missing check for scalars; won't choke on list/dict
    return v is None or v is pd.NA or (isinstance(v, float) and np.isnan(v))

def save_to_csv(df_in: pd.DataFrame, fname: str):
    out = df_in.copy()

    json_cols = ["kwic_tokens", "target_sentence_tokens"]
    for c in json_cols:
        if c not in out.columns:
            continue
        out[c] = out[c].map(
            lambda v: ""
            if _is_missing(v)
            else json.dumps(v, ensure_ascii=False, separators=(",", ":"))
            if isinstance(v, (list, dict))
            else str(v)
        )

    # GitHub CSV preview killer: literal newlines inside a cell
    obj_cols = out.select_dtypes(include=["object", "string"]).columns
    for c in obj_cols:
        out[c] = out[c].astype(str).str.replace("\r", "\\r", regex=False).str.replace("\n", "\\n", regex=False)

    out.to_csv(
        os.path.join(csv_path, fname.replace(".parquet", ".csv")),
        sep=",",                 # <- important for GitHub preview
        index=False,
        encoding="utf-8",
        lineterminator="\n",
        quoting=csv.QUOTE_ALL,
        escapechar="\\",
        doublequote=True,
    )

def read_and_filter_hits(fname):
    try:
        df = pd.DataFrame(read_hits(fname))

        grela = df["grela_id"].astype("string")

        nos_mask = grela.str.startswith("noscemus", na=False) & ~grela.str[9:].isin(noscemus_emlap_ids)
        noscemus_instances_N = int(nos_mask.sum())

        cc_instances_N = int(grela.str.startswith("cc", na=False).sum())

        emlap_instances_df = df[grela.str.startswith("emlap", na=False)].copy()

        if not emlap_instances_df.empty:
            save_to_csv(emlap_instances_df, fname)
            emlap_instances = emlap_instances_df.to_dict("records")
            emlap_instances_N = len(emlap_instances)
        else:
            emlap_instances = []
            emlap_instances_N = 0

        return {
            "noscemus_instances_N": noscemus_instances_N,
            "cc_instances_N": cc_instances_N,
            "emlap_instances_N": emlap_instances_N,
            "emlap_instances": emlap_instances,
        }
    except Exception:
        return {"noscemus_instances_N": 0, "cc_instances_N": 0, "emlap_instances_N": 0, "emlap_instances": []}

In [28]:
%%time
instances_dict_series = pd.DataFrame(lexeme_df["instance_fname"].apply(read_and_filter_hits).tolist())

CPU times: user 5min 21s, sys: 36 s, total: 5min 57s
Wall time: 5min 29s


In [29]:
lexeme_df = lexeme_df.reset_index(drop=True)
lexeme_merged_df = lexeme_df.join(instances_dict_series)

In [31]:
lexeme_merged_df[lexeme_merged_df["target_relemmatized"].apply(lambda x: len(x.split()) > 1)]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,...,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized,instance_fname,noscemus_instances_N,cc_instances_N,emlap_instances_N,emlap_instances
23,Lapis maior,[Lapis maior],lapis maior,"[Ruland1612-Lapis-maior, Ruland1612-Lapis-maio...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Lapis-m...",3,...,"[None, None, id est, der Geist außgezogen auß ...","[finde nach etlicher Meynung, die calcinirten ...","[{'entry_id': 'Ruland1612-Lapis-maior', 'quote...",2,lapis magnus,lapis_maior.parquet,47,222,29,"[{'target_phrase': ['lapidem', 'magnum'], 'tar..."
24,Leo viridis,[Leo viridis],leo uiridis,"[Ruland1612-Leo-viridis, Ruland1612-Leo-viridi...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Leo-vir...",3,...,"[None, quorundam opinione aurum, vitriolum]","[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[{'entry_id': 'Ruland1612-Leo-viridis', 'quote...",2,leo uiridis,leo_uiridis.parquet,17,8,45,"[{'target_phrase': ['leo', 'uiridis'], 'target..."
25,Machina tractoria,[Machina tractoria],machina tractoria,"[Ruland1612-Machina-tractoria, Ruland1612-Mach...",3,[M],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Machina...",3,...,"[None, None, None]","[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[{'entry_id': 'Ruland1612-Machina-tractoria', ...",2,machina tractoria,machina_tractoria.parquet,15,0,0,[]
30,Panum aereorum species,[Panum aereorum species],panum aereorum species,"[Ruland1612-Panum-aereorum-species-,-Vorpleits...",3,[P],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Panum-a...",3,...,"[None, None, None]","[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...",[{'entry_id': 'Ruland1612-Panum-aereorum-speci...,3,panis aereus species,panum_aereorum_species.parquet,3,0,0,[]
32,Pyritae aurei coloris,[Pyritae aurei coloris],pyritae aurei coloris,[Ruland1612-Pyritae-aurei-coloris-.-1.-Pyrites...,3,[P],[],[],"[{'def': '1', 'entry_id': 'Ruland1612-Pyritae-...",3,...,"[1, è quibus ignis elicitur, è quibus ignis no...",[1. Pyrites qui est solidus & colorem auri tot...,[{'entry_id': 'Ruland1612-Pyritae-aurei-colori...,3,pyritus aureus color,pyritae_aurei_coloris.parquet,5,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840,Fibula ferrea,[Fibula ferrea],fibula ferrea,[Ruland1612-Fibula-ferrea],1,[F],[],[],[],0,...,[],[],"[{'entry_id': 'Ruland1612-Fibula-ferrea', 'quo...",2,fibula ferreus,fibula_ferrea.parquet,35,2,2,"[{'target_phrase': ['fibulis', 'ferreis'], 'ta..."
2841,Leux amygdala,[Leux amygdala],leux amygdala,[Ruland1612-Leux-amygdala.],1,[L],[],[],[],0,...,[],[],[],2,leux amygdalum,leux_amygdala.parquet,1,0,0,[]
2842,Lignum crucis,[Lignum crucis],lignum crucis,[Ruland1612-Lignum-crucis.],1,[L],[],[],[],0,...,[],[],[],2,lignum crux,lignum_crucis.parquet,18,940,0,[]
2843,Minera vena terrae,[Minera vena terrae],minera uena terrae,"[Ruland1612-Minera-vena-terrae-,-Ein-Erz-Ader.]",1,[M],[],[],[],0,...,[],[],"[{'entry_id': 'Ruland1612-Minera-vena-terrae-,...",3,minera uena terra,minera_uena_terrae.parquet,1,0,0,[]


In [32]:
lexeme_merged_df.sort_values("emlap_instances_N", ascending=False).head(20)

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,...,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized,instance_fname,noscemus_instances_N,cc_instances_N,emlap_instances_N,emlap_instances
596,Aurum,[Aurum],aurum,[Ruland1612-Aurum],1,[A],[],"[Natiuum purum, quod a natura tale est, cuius ...","[{'def': 'à Germanis', 'entry_id': 'Ruland1612...",1,...,[à Germanis],"[, à Germanis Goldtsa Chymistis dicitur Sol, &...","[{'entry_id': 'Ruland1612-Aurum', 'quote': 'Go...",1,aurum,aurum.parquet,9214,37216,11585,"[{'target_phrase': ['auro'], 'target_from': 'c..."
526,Argentum,[Argentum],argentum,"[Ruland1612-Argentum-à-Chymistis-Luna,-cui-eti...",1,[A],[Argentum],[],"[{'def': 'à Chymistis Luna, cui etiam tribuitu...",1,...,"[à Chymistis Luna, cui etiam tribuitur]","[à Chymistis Luna, cui etiam tribuitur: estque...",[{'entry_id': 'Ruland1612-Argentum-à-Chymistis...,1,argentum,argentum.parquet,7566,23030,10388,"[{'target_phrase': ['argento'], 'target_from':..."
29,Oleum,[Oleum],oleum,"[Ruland1612-Oleum, Ruland1612-Oleum, Ruland161...",3,[O],[],[],"[{'def': 'id est, ignis, wirdt außgezogen von ...",3,...,"[id est, ignis, wirdt außgezogen von einer tro...","[id est, ignis, wirdt außgezogen von einer tro...","[{'entry_id': 'Ruland1612-Oleum', 'quote': 'wi...",1,oleum,oleum.parquet,12829,28096,10016,"[{'target_phrase': ['oleo'], 'target_from': 'c..."
2277,Sal,[Sal],sal,[Ruland1612-Sal],1,[S],[],[],"[{'def': 'Alas Graece, Melech Arabice, vel sel...",1,...,"[Alas Graece, Melech Arabice, vel selenipum es...","[Alas Graece, Melech Arabice, vel selenipum es...","[{'entry_id': 'Ruland1612-Sal', 'quote': 'Salt...",1,sal,sal.parquet,12636,12520,9028,"[{'target_phrase': ['sale'], 'target_from': 'c..."
96,Metallum,[Metallum],metallum,"[Ruland1612-Metallum, Ruland1612-Metallum]",2,[M],[],[],"[{'def': 'vena, Ertz.', 'entry_id': 'Ruland161...",2,...,"[vena, Ertz., id est, conflatum argentum.]","[vena, Ertz., id est, conflatum argentum.]","[{'entry_id': 'Ruland1612-Metallum', 'quote': ...",1,metallum,metallum.parquet,7323,5847,9026,"[{'target_phrase': ['metallis'], 'target_from'..."
27,Mercurius,[Mercurius],mercurius,"[Ruland1612-Mercurius, Ruland1612-Mercurius, R...",3,[M],[],[],"[{'def': 'id est, sulphur.', 'entry_id': 'Rula...",3,...,"[id est, sulphur., est principium materiale, v...","[id est, sulphur. Mercurius. Mercurius ist in ...","[{'entry_id': 'Ruland1612-Mercurius', 'quote':...",1,mercurius,mercurius.parquet,8729,4942,7928,"[{'target_phrase': ['mercurius'], 'target_from..."
10,Sulphur,[Sulphur],sulphur,"[Ruland1612-Sulphur, Ruland1612-Sulphur, Rulan...",4,[S],[],[],"[{'def': 'chibur, vel, Albusao Arabice dicitur...",4,...,"[chibur, vel, Albusao Arabice dicitur, pars la...","[chibur, vel, Albusao Arabice dicitur, pars la...","[{'entry_id': 'Ruland1612-Sulphur', 'quote': '...",1,sulphur,sulphur.parquet,4367,2719,6960,"[{'target_phrase': ['sulphur'], 'target_from':..."
1200,Elementum,[Elementum],elementum,[Ruland1612-Elementum],1,[E],[],[],[{'def': 'quicquid ex matricibus elementorum p...,1,...,"[quicquid ex matricibus elementorum prouenit,]","[quicquid ex matricibus elementorum prouenit, ...","[{'entry_id': 'Ruland1612-Elementum', 'quote':...",1,elementum,elementum.parquet,9276,23583,6959,"[{'target_phrase': ['elementa'], 'target_from'..."
18,Essentia,[Essentia],essentia,"[Ruland1612-Essentia, Ruland1612-Essentia, Rul...",3,[E],[],[],[{'def': 'est extractum simplex e reb. tota su...,3,...,[est extractum simplex e reb. tota sua natura ...,[est extractum simplex e reb. tota sua natura ...,[],1,essentia,essentia.parquet,5000,54270,5330,"[{'target_phrase': ['essentia'], 'target_from'..."
197,Aer,[Aer],aer,[Ruland1612-Aer],1,[A],[],[],"[{'def': 'id est, aura, halitus, spiritus, ven...",1,...,"[id est, aura, halitus, spiritus, ventus,]","[id est, aura, halitus, spiritus, ventus, Wett...","[{'entry_id': 'Ruland1612-Aer', 'quote': 'Wett...",1,aer,aer.parquet,27774,43181,51

In [35]:
lexeme_df = lexeme_merged_df

In [36]:
#emlap_id_title_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['working_title']))
#emlap_id_date_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['date_publication']))

In [37]:
#def add_emlap_metadata(instances_dict_list):
#    instances_dict_list_new = []
#    for instance_dict in instances_dict_list:
#        id = instance_dict["grela_id"][6:]
#        instance_dict["title"] = emlap_id_title_dict[id]
#        instance_dict["year"] = emlap_id_date_dict[id]
#        instances_dict_list_new.append(instance_dict)
#    return instances_dict_list_new

In [39]:
# conservative under Google’s 50k hard limit
MAX_CELL = 48000

def _clip(s: str, n: int = MAX_CELL) -> str:
    return s if len(s) <= n else (s[:n-1] + "…")

def _stringify(v) -> str:
    """Controlled stringification so lists/dicts don't explode."""
    if v is None or (isinstance(v, float) and np.isnan(v)):
        return ""
    if isinstance(v, (list, dict)):
        # compact JSON, unicode-friendly
        return json.dumps(v, ensure_ascii=False, separators=(',', ':'))
    return str(v)

def prepare_for_sheets(df: pd.DataFrame, *, drop_heavy: bool = False) -> pd.DataFrame:
    """
    Make a sheets-safe copy:
      - optional: drop very heavy/nested cols
      - stringify lists/dicts in a compact way
      - clip any cell > MAX_CELL
    """
    if drop_heavy:
        df = df.drop(columns=[
            # keep this list lean; tweak as you like
            "senses_flat", "Senses", "Translations", "EntryLevelTranslations"
        ], errors="ignore").copy()
    else:
        df = df.copy()

    # stringify then clip per-cell
    df = df.map(_stringify)
    df = df.map(lambda s: _clip(s, MAX_CELL))
    return df

# --- example usage ---
# keep everything but serialize/clip:
df_for_sheet = prepare_for_sheets(lexeme_df.drop(["emlap_instances"], axis=1), drop_heavy=False)

In [40]:
set_with_dataframe(ruland_gs.add_worksheet("ruland_emlap_instances_2025-12-16", 1,1), df_for_sheet.sort_values("emlap_instances_N", ascending=False))

In [41]:
lexeme_df.to_json("../data/large_files/ruland-emlap-grela.json")

In [42]:
lexeme_df.to_parquet("../data/large_files/ruland-emlap-grela.parquet")